In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/Users/parthshukla/Documents/CollabProject/NBA-BoxScores-2024-2025.csv')
df.head()

,Unnamed: 0,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0,22400061,1610612752,NYK,New York,1628384,OG Anunoby,OG,F,NaN,...,0.0,5.0,5.0,3.0,1.0,0.0,0.0,3.0,4.0,-21.0
1,1,22400061,1610612752,NYK,New York,1628404,Josh Hart,Josh,F,NaN,...,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,12.0,-23.0
2,2,22400061,1610612752,NYK,New York,1626157,Karl-Anthony Towns,Karl-Anthony,C,NaN,...,0.0,7.0,7.0,3.0,0.0,0.0,0.0,1.0,12.0,-18.0
3,3,22400061,1610612752,NYK,New York,1628969,Mikal Bridges,Mikal,G,NaN,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,16.0,-33.0
4,4,22400061,1610612752,NYK,New York,1628973,Jalen Brunson,Jalen,G,NaN,...,0.0,1.0,1.0,2.0,0.0,0.0,4.0,3.0,22.0,-23.0
